In [114]:
import torch
import torch.optim as optim
import torch.cuda.amp as amp
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split

import torchvision
from torchvision import models, transforms, datasets

from PIL import Image
import argparse

# hyperparameter
T = 2.0
student_model = 18

batch_size = 1024
num_workers = 4

epochs = 25
alpha = 0.7
student = 0

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.244, 0.225]),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(5)
])
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# teacher = ResNet50
teacher = models.resnet50(pretrained=True).to(device).eval()

# student = ResNet18, ResNet34, ResNet50
if student_model == 18:
    student = models.resnet18(pretrained=False).to(device)
elif student_model == 34:
    student = models.resnet34(pretrained=False).to(device)
elif student_model == 50:
    student = models.resnet50(pretrained=False).to(device)
    
imagenet = datasets.ImageNet('./data/datasets/ImageNet/train/', split='train', transform=transform)
testset = datasets.ImageNet('./data/datasets/ImageNet/val/', split='val', transform=transform)
trainset, _ = random_split(imagenet, [1000000, len(imagenet) - 1000000])

trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=True, drop_last=True)
testloader = DataLoader(testset, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=True, drop_last=True)

RuntimeError: The archive ILSVRC2012_devkit_t12.tar.gz is not present in the root directory or is corrupted. You need to download it externally and place it in ./data/datasets/ImageNet/train/.

In [87]:
def DistillationLoss(teacher_logit, student_logit, T):
    soft_label = F.softmax(teacher_logit / T, dim=1)[0]
    soft_prediction = F.log_softmax(student_logit / T, dim=1)[0]
    return nn.KLDivLoss()(soft_prediction, soft_label)

def StudentLoss(student_logit, hard_label):
    hard_label = F.softmax(student_logit, dim=1)[0]
    return nn.CrossEntropyLoss()(student_logit, hard_label)

def FinalLoss(teacher_logit, student_logit, labels, T, alpha):
    return (1. - alpha) * StudentLoss(student_logit, labels) \
            + (alpha * T * T) * DistillationLoss(teacher_logit, student_logit, T)

optimizer = optim.Adam(student.parameters(), lr=0.001)
scaler = amp.GradScaler()

In [88]:
print(f'Config : Student = ResNet{student_model}, T = {T}, alpha = {alpha}')

for epoch in range(epochs):
    train_loss = 0.0
    train_acc = 0.0
    train_samples = 0
    
    for inputs, labels in trainloader:
        inputs, labels = inputs.to(device), labels.to(device)

        with amp.autocast():
            student_logit = student(inputs)
            teacher_logit = teacher(inputs)
            loss = FinalLoss(teacher_logit, student_logit, labels, T, alpha)
        
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        _, preds = torch.max(outputs, 1)
        train_loss += loss.item()
        train_acc += torch.sum(preds == labels.data)
        train_samples += len(inputs)
    
    epoch_loss = train_loss / len(trainloader)
    epoch_acc = train_acc.float() / train_samples * 100
    
    if (epoch + 1) % 5 == 0 or epoch == 0:
        print(f"epoch: {epoch + 1} || tl: {epoch_loss:.3f}, ta: {epoch_acc:.2f}%")

print('Training Finished')

correct_s, correct_5s, total_s = 0, 0, 0
correct_t, correct_5t, total_t = 0, 0, 0

with torch.no_grad():
    for images, labels in testloader:
        images, labels = images.to(device), labels.to(device)
        
        # student - top1
        outputs_s = student(images)
        _, predicted_s = torch.max(outputs_s.data, 1)
        total_s += labels.size(0)
        correct_s += (predicted_s == labels).sum().item()
        # top5
        for idx, item in enumerate(labels.view(-1,1)):
            if item in torch.topk(outputs_s, 5).indices[idx]:
                correct_5s += 1
        
        # teacher - top1
        outputs_t = teacher(images)
        _, predicted_t = torch.max(outputs_t.data, 1)
        total_t += labels.size(0)
        correct_t += (predicted_t == labels).sum().item()
        # top5
        for idx, item in enumerate(labels.view(-1,1)):
            if item in torch.topk(outputs_t, 5).indices[idx]:
                correct_5t += 1

print(f'Top1 Acc : student - {correct_s*100/total_s:0.1f}% / teacher - {correct_t*100/total_t:0.1f}%')
print(f'Top5 Acc : student - {correct_5s*100/total_s:0.1f}% / teacher - {correct_5t*100/total_t:0.1f}%')
print('===========================================')

Config : Student = ResNet18, T = 2.0, alpha = 0.7


NameError: name 'trainloader' is not defined

In [ ]:
PATH = './student_' + str(student_model) + '_' + str(T) + '.pth'
torch.save(net.state_dict(), PATH)

In [134]:
label = torch.Tensor([3, 5, 3])

x = torch.Tensor([[.1,.2,.3,.4,.5,.6],
                 [.7,.5,.2,.3,.4,.1],
                 [.5,.6,.8,.3,.7,.9]])
print(x)
label.view(-1,1) in torch.topk(x, 3).indices

tensor([[0.1000, 0.2000, 0.3000, 0.4000, 0.5000, 0.6000],
        [0.7000, 0.5000, 0.2000, 0.3000, 0.4000, 0.1000],
        [0.5000, 0.6000, 0.8000, 0.3000, 0.7000, 0.9000]])


True

In [135]:
correct = 0

for idx, item in enumerate(label.view(-1,1)):
    if item in torch.topk(x, 3).indices[idx]:
        correct += 1


In [138]:
((label.view(-1,1) in torch.topk(x, 3).indices) == True)

True